In [1]:
import requests 

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')


index_name = "new-course-questions"

if not es_client.indices.exists(index=index_name):
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    es_client.indices.create(index=index_name, body=index_settings)
    
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
        
    for course in documents_raw:
        course_name = course['course']
    
        for doc in course['documents']:
            doc['course'] = course_name
            es_client.index(index=index_name, document=doc)

In [3]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [4]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text'] + "\n\n")

    return prompt_template.format(question=query, context=context).strip()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

query = "How do I execute a command in a running docker container?"

search_results = elastic_search(query)

prompt = build_prompt(query, search_results)
print(len(prompt))

result = enc.encode(prompt)
print(len(result))

1322
298
